In [2]:
import pandas as pd
import numpy as np
import google.datalab.bigquery as bqd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
import sklearn.metrics

## Get data from Datacamp's S3 bucket

In [3]:
df = pd.read_csv("https://s3.amazonaws.com/assets.datacamp.com/blog_assets/fake_or_real_news.csv")
print(df.shape)
print("Number of fakes: ",df[df['label'].str.contains("FAKE")].shape[0])
df.head()

(6335, 4)
Number of fakes:  3164


,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


For now, only text is used as a feature until title is explored differently

In [4]:
# Do a 80-20 split of data into train and test sets. 
df = df[['title','text','label']]
trainX, testX, trainY, testY = train_test_split(df['text'], df.index, test_size = 0.20, random_state=44)

In [14]:
# Initialize vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words = 'english')
train_tfidf = tfidf_vectorizer.fit_transform(trainX)
test_tfidf = tfidf_vectorizer.transform(testX)

/usr/local/envs/py3env/lib/python3.5/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [24]:
type(train_tfidf[:10,:])

scipy.sparse.csr.csr_matrix

In [20]:
#tfidf_vectorizer.get_feature_names()[-10:]
tfidf_df = pd.DataFrame(train_tfidf.A, columns=tfidf_vectorizer.get_feature_names())
tfidf_df.head()

,00,000,0000,000000031,000035,0001,0002,000billion,000ft,000km,...,تنجح,حلب,عربي,عن,لم,ما,محاولات,من,هذا,والمرضى
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    See full source and example: 
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
    
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [32]:
clf = MultinomialNB()

In [ ]:
train_tfidf[:100].shape

In [33]:
clf.fit(train_tfidf[:100], trainY[:100])
pred = clf.predict(test_tfidf)
print("Accuracy: ", sklearn.metrics.accuracy_score(testY, pred))

cm = sklearn.metrics.confusion_matrix(testY, pred, labels=['FAKE','REAL'])
plot_confusion_matrix(cm, classes=['FAKE','REAL'])

Accuracy:  0.0


/usr/local/envs/py3env/lib/python3.5/site-packages/sklearn/metrics/classification.py:248: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if np.all([l not in y_true for l in labels]):


ValueError: At least one label specified must be in y_true

In [9]:
sampleDF.to_gbq('FakeNews.SampleTable','vu-nlp',verbose=True)




Load is 100% Complete




In [2]:
def load_sample():
  q = bqd.Query(sql='''
    SELECT
      title
    FROM
      `vu-nlp.FakeNews.FAKENEWS` 
  WHERE
    country = 'US'
    and language ='english'
  LIMIT 10
  ''')
  
  print("Executing BigQuery query...")
  return(q.execute().result().to_dataframe())

sampleDF = load_sample()
sampleDF

NameError: global name 'bqd' is not defined